_Imports we'll need for later:_

In [ ]:
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import biff
import gala.dynamics as gd
import gala.potential as gp
import gala.integrate as gi
from gala.units import galactic
from scipy.optimize import minimize, root
from scipy.misc import derivative

First some background / context. For Milky Way science, we commonly use a cartesian coordinate system aligned with the plane of the disk of our galaxy. By convention, the sun is typically put on the $x$-axis of this system, with the Galactic center ~8 kpc away along this axis. The $y$-axis is defined to be in the direction of the sun's orbit. Here's a cartoon of this system, looking down on the plane of the Galaxy with the approximate location of the sun indicated by the $\odot$ symbol:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.text(-8, 0, r'$\odot$', fontsize=18, ha='center', va='center', zorder=100)

kw = dict(arrowprops=dict(arrowstyle="->", edgecolor='#555555', shrinkA=0, shrinkB=0), 
          zorder=10)
offset = 4.
ax.annotate("", xy=(-8+offset, 0.), xytext=(-8, 0), **kw)
ax.annotate("", xy=(-8, offset), xytext=(-8, 0), **kw)

ax.text(-8+offset+offset/10, 0, '$x$', ha='left', va='center', fontsize='20')
ax.text(-8, offset+offset/10, '$y$', ha='center', va='bottom', fontsize='20')

circ = mpl.patches.Circle((0,0), radius=15, zorder=-100, color='tab:blue', alpha=0.1)
ax.add_patch(circ)

ax.text(1, 0., 'galactic center', fontsize=14, va='center')
ax.scatter(0, 0, marker='x', linewidth=2., s=40, c='k')

ax.set_xlim(-16, 16)
ax.set_ylim(-16, 16)
ax.set_aspect('equal', adjustable='box')

## Integrating orbits in a rotating frame:

To do this, we have to specify the rotation frequency and axis of rotation of the rotating frame. In all of our applications, we'll set the frame rotation equal to the pattern speed of the bar model we use, and we'll always assume rotation is around the Galactic $z$ axis. Let's define one such frame, assuming a pattern speed $\Omega_p = 40~{\rm km}~{\rm s}^{-1}~{\rm kpc}^{-1}$, and we'll generally work in the "galactic" unit system (imported above: kpc, Myr, solar masses, and radians):

In [ ]:
galactic

In [ ]:
Om_p = 40 * u.km/u.s / u.kpc
frame = gp.ConstantRotatingFrame([0,0,-1] * Om_p, units=galactic)

Now let's construct a simple bar model using a model with an analytic density and potential form. This was first described in a paper by Long & Murali, so the potential class is `LongMuraliBarPotential`. The parameters are:
* `m` - total mass
* `a` - major axis scale length ($x$ axis in our case)
* `b` - intermediate axis scale length ($y$ axis in our case)
* `c` - minor axis scale length  ($z$ axis in our case)
* `alpha` - angle of the bar w.r.t. $x$-axis of rotating frame (default = 0)

We'll always use a bar angle $\alpha = 0$:

In [ ]:
bar = gp.LongMuraliBarPotential(m=1E10*u.Msun, 
                                a=1.49*u.kpc, b=0.58*u.kpc, c=0.4*u.kpc, 
                                units=galactic)

Let's plot isopotential and isodensity contours for this choice of parameters. We can use the `plot_contours()` and `plot_density_contours()` methods of the potential object. We have to specify the grid to compute the value of the potential over. If we specify a 2D grid, we then have to set the value of $z$ to slice at - here we'll use the same grid in $x$ and $y$, and slice at $z=0$:

In [ ]:
xy = np.linspace(-20, 20, 128)
grid = (xy, xy, 0)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
_ = bar.plot_contours(grid, ax=axes[0])
_ = bar.plot_density_contours(grid, ax=axes[1])

Let's try changing the bar scale lengths and see what happens:

In [ ]:
bar2 = gp.LongMuraliBarPotential(m=1E10*u.Msun, 
                                 a=3*u.kpc, b=1*u.kpc, c=0.2*u.kpc, 
                                 units=galactic)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
_ = bar2.plot_contours(grid, ax=axes[0])
_ = bar2.plot_density_contours(grid, ax=axes[1])

Let's switch back to use our original model.

With the bar potential model object, we can compute various quantities. For example, we can estimate the total mass enclosed within a spherical radius:

In [ ]:
grid = np.linspace(0.1, 25, 128)
xyz = np.zeros((3, grid.size))
xyz[1] = grid

Menc = bar.mass_enclosed(xyz * u.kpc)
plt.plot(grid, Menc, marker='')
plt.xlabel('$r$ [kpc]')
plt.ylabel('$M(<r)$ [{0:latex}]'.format(u.Msun))

Of course, the bar is only one component of the Galaxy: the total gravitational potential will be a mixture of the bar, the disk, and the dark matter halo. Let's construct a simple model for the total potential of the Milky Way. We think that the circular velocity curve (the velocity of a circular orbit at a given radius) is close to flat over a large range of radii in the Milky Way, and is around 220 km/s at the sun's location.

In [ ]:
bar = gp.LongMuraliBarPotential(m=5E9*u.Msun, 
                                a=1.49*u.kpc, b=0.58*u.kpc, c=0.4*u.kpc, 
                                units=galactic)
disk = gp.MiyamotoNagaiPotential(m=4.5E10*u.Msun, a=2.5, b=0.28, units=galactic)
halo = gp.NFWPotential(m=6E11*u.Msun, r_s=15*u.kpc, units=galactic)
milkyway = bar + halo + disk

Let's look at the circular velocity curve and see if it's ~220 km/s at the Sun's radius:

In [ ]:
grid = np.linspace(0.1, 50, 128)
xyz = np.zeros((3, grid.size))
xyz[0] = grid

vcirc = milkyway.circular_velocity(xyz * u.kpc)
plt.plot(grid, vcirc, marker='')
plt.axhline(220)
plt.axvline(8)
plt.xlabel('$r$ [kpc]')
plt.ylabel('$v_c(r)$ [{0:latex_inline}]'.format(u.km/u.s))

Except for the bar, all of the other components of our Galaxy model are either close to spherical or axisymmetric, so when we integrate orbits in the rotating frame, the bar is the only non-axisymmetric feature. Let's now construct a Hamiltonian object with our Milky Way model and the rotating frame we defined early on:

In [ ]:
H = gp.Hamiltonian(milkyway, frame=frame)

Let's look at the orbit of the Sun in this potential, which has a total velocity around $\boldsymbol{v} \approx (11, 240, 7)~{\rm km}~{\rm s}^{-1}$:

In [ ]:
sun = gd.PhaseSpacePosition(pos=[-8., 0, 0]*u.kpc,
                            vel=[11, 240, 7.]*u.km/u.s)

In [ ]:
orbit = H.integrate_orbit(sun, dt=0.5*u.Myr, n_steps=2000)
_ = orbit.plot(['x', 'y'])

The above is what the orbit looks like in the *rotating* frame. We can transform it to an inertial frame and plot it there instead:

In [ ]:
static_frame = gp.StaticFrame(galactic)

In [ ]:
orbit_inertial = orbit.to_frame(static_frame)
_ = orbit_inertial.plot(['x', 'y'])

Let's do the same for 4 randomly sampled orbits near the Sun's postion:

In [ ]:
np.random.seed(101)
new_pos = np.random.normal(sun.xyz, 1., size=(4, 3)).T * sun.xyz.unit
new_vel = np.random.normal(sun.v_xyz, 25, size=(4, 3)).T * sun.v_xyz.unit
near_sun = gd.PhaseSpacePosition(new_pos, new_vel)

orbits = H.integrate_orbit(near_sun, dt=0.5*u.Myr, n_steps=8000)

fig, axes = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
for n in range(orbits.xyz.shape[-1]):
    _ = orbits[:, n].plot(['x', 'y'], axes=[axes.flat[n]], )
    
axes.flat[0].set_xlim(-15, 15)
axes.flat[0].set_ylim(-15, 15)

Notice that some orbits fill what looks like an annular ring, whereas others look like they just trace out lines (e.g., upper left panel). Orbits that fill an annulus are normal, non-resonant orbits with finite eccentricity. The eccentricities mean that the orbits oscillate in the radial direction while they circulate in azimuth. 

Any orbit with small excursions in the $z$ direction can be mostly characterized in terms of its frequency of azimuthal rotation, $\Omega_\phi$, and its radial oscillation frequency, $\Omega_r$. Most orbits have similar values for these frequencies (within a factor of a few), but the values are otherwise not related. Orbits like the one in the upper left are resonant orbits. For resonant orbits, the frequencies conspire so that the azimuthal frequency, the radial frequency, and the bar frequency all have some integer relation. There are a few types of well-known resonant orbits in rotating potentials, the easiest conceptually is the _corotation resonance_, in which:
$$
\Omega_\phi = \Omega_b
$$
Right on the resonance, an orbit will appear as a dot in the rotating frame: it has the same frequency as the frame rotation, so from the perspective of someone in the rotating frame, the orbit doesn't move! Close to the corotation resonance, orbits trace out little paths, but are generally unstable. Let's try to create and integrate an orbit close to corotation.

To do this, we can compute the circular frequency curve: the circular velocity curve divided by radius.

In [ ]:
grid = np.linspace(1, 10, 128)
xyz = np.zeros((3, grid.size))
xyz[0] = grid

circ_freq = (milkyway.circular_velocity(xyz * u.kpc) / (grid * u.kpc)).to(u.km/u.s/u.kpc)
plt.plot(grid, circ_freq, marker='')
plt.axhline(np.abs(frame.parameters['Omega'][2].to(u.km/u.s/u.kpc).value))
plt.xlabel('$r$ [kpc]')
plt.ylabel(r'$\Omega(r)$ [{0:latex_inline}]'.format(u.km/u.s/u.kpc))

To find the precise value, we can use `scipy.optimize.minimize` to find the radius where the circular orbit frequency and the pattern speed match:

In [ ]:
def Omega_phi(R):
    return (milkyway.circular_velocity([R, 0, 0] * u.kpc) / (R * u.kpc)).to(u.km/u.s/u.kpc)[0]

In [ ]:
def func(p):
    r = p[0]
    return (Omega_phi(r) - Om_p).value**2

res = minimize(func, x0=10., tol=1E-11)
corot_r = res.x[0]

In [ ]:
xyz = [-corot_r, 0, 0] * u.kpc
vc = milkyway.circular_velocity(xyz)
corot_w0 = gd.PhaseSpacePosition(pos=xyz,
                                 vel=[0, vc.value, 0] * vc.unit)

In [ ]:
orbit = H.integrate_orbit(corot_w0, dt=0.5, n_steps=10000)

fig, ax = plt.subplots(1, 1, figsize=(5.5, 5))
fig = orbit.plot(['x', 'y'], marker='o', ms=5, axes=[ax])
ax.set_xlim(-8, 8)
ax.set_ylim(-8, 8)

Another set of famous rotating frame resonances are the _Lindblad resonances_. These are orbits that have an integer relation between the pattern speed, radial frequency, and azimuthal frequency. These orbits look pretty cool, but are hard to find in complex potential models like the one we're using. To find these, we need to find a radius at which:
$$
m\,(\Omega_\phi - \Omega_b) = \pm \Omega_r
$$

It's harder to estimate $\Omega_r$, but you could do it numerically by computing the 2nd derivative of the potential around some position (see eq. 3.146b in Binney and Tremaine).

## Exercise for you:

Integrate some random orbits in the barred Milky Way model we've been using in this notebook and try to find some other examples of resonant orbits. I'd recommend keeping the position of the initial conditions fixed at some location in the xy plane (set z = vz = 0), and vary the vx and vy components. Once you find one, plot it in the rotating frame and in a static frame.